In [5]:
pip install langchain langchain-text-splitters

Note: you may need to restart the kernel to use updated packages.


In [12]:
with open("../data/actionplan - whole.txt", 'r') as file:
    text = file.read()


import re
import uuid
import tiktoken

summary = { 
    "topic" : { 
        "act_plan_count": "",
        "act_plan_list": [],
        "distinct_count": "",
        "distinct_list": []
},
    "gene" : {
        "act_plan_count": "",
        "act_plan_list": [],
        "distinct_count": "",
        "distinct_list": []
    }
}

def topic_chuncker(text):
    topic_arr = []
    display   = [] 
    pattern = r"Topic:\s*(.*?)\n(.*?)(?=\nTopic:|\Z)"
    
    topic_match = re.findall(pattern, text, re.DOTALL)
    
    for topic, context in topic_match:

        temp_topic = { 
                      "topic": topic.strip(),
                      "content": context.strip()
                      }
        display.append(temp_topic["topic"])
        topic_arr.append(temp_topic)
    
    distinct_set = set(display)
    
    
    # SUMARY
    summary["topic"]["act_plan_count"] = len(topic_arr)
    summary["topic"]["act_plan_list"]  = display
    summary["topic"]["distinct_count"] = len(distinct_set)
    summary["topic"]["distinct"]       = list(distinct_set)
    print ( len( topic_match ), " topics found")
    return topic_arr

def gene_chunker(): 
    gene_arr = []
    display = []
    topic_chunks = topic_chuncker(text)
    
    for topic in topic_chunks:
        topic_name = topic["topic"]
        content = topic["content"]
        
        pattern = r"Gene:\s*(.*?)\n(.*?)(?=\nGene:|\Z)"
        gene_match = re.findall(pattern, content, re.DOTALL)
        for gene, gene_content in gene_match:
            if gene in content:
                temp_topic_gene = {
                    "topic": topic_name,
                    "gene": gene.strip(),
                    "content": gene_content.strip()
                }
                display.append(temp_topic_gene["gene"]) 
                gene_arr.append(temp_topic_gene)
    distinct_set = set(display)
    
    # SUMARY
    summary["gene"]["act_plan_count"] = len(gene_arr)
    summary["gene"]["act_plan_list"] = display
    summary["gene"]["distinct_count"] = len(distinct_set)
    summary["gene"]["distinct_list"] = list(distinct_set)
    print(len(distinct_set), " genes found" )
    return gene_arr

def age_chunker():
    age_arr = []
    gene_chunks = gene_chunker()

    for gene in gene_chunks:
        topic = gene["topic"]
        genes  = gene["gene"]
        content = gene["content"]
        
        pattern = r"Min Age:\s*(\d+)\s*Max Age:\s*(\d+)\s*(.*?)(?=\nMin Age:|\Z)"
        age_match = re.findall(pattern, content, re.DOTALL)
        if age_match:
            for min_age, max_age, age_content in age_match:
                if min_age in content or max_age in content:
                    temp_topic_age = {
                        "topic": topic,
                        "gene": genes,
                        "min_age": min_age.strip(),
                        "max_age": max_age.strip(),
                        "content": age_content.strip()
                    }
                age_arr.append(temp_topic_age)
        
    return age_arr

def num_tokens_from_string(string: str, encoding_name: str = "cl100k_base") -> int:
    """Returns the number of tokens in a text string."""
    """"r50k_base", "p50k_base", "cl100k_base", "o200k_base"""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

def heading_chunker():
    heading_arr = []
    age_chunks = age_chunker()
    
    
    for age in age_chunks:
        age_name = age["topic"]
        gene     = age["gene"]
        min_age  = age["min_age"]
        max_age  = age["max_age"]
        content  = age["content"]
        
        pattern = r"Heading:\s*(.*?)\n(.*?)(?=\nHeading:|\Z)"

        heading_match = re.findall(pattern, content, re.DOTALL)
        
        for heading, heading_content in heading_match:
            # print( "----------------------------------------------")
            # print("heading -> ", heading)
            # print("content -> ", content)
            if heading in content:
                temp_topic_heading = {
                    "id": str( uuid.uuid4() ),
                    "topic": age_name,
                    "gene": gene,
                    "min_age": min_age,
                    "max_age": max_age,
                    "token": num_tokens_from_string(heading_content.strip()),
                    "heading": heading.strip(),
                    "content": heading_content.strip()
                }
                heading_arr.append(temp_topic_heading)
     
    return heading_arr


rs = heading_chunker()
print("summary -> ", summary)
print("rs -> ", rs )

21  topics found
4  genes found
summary ->  {'topic': {'act_plan_count': 21, 'act_plan_list': ['Welcome to your ActionPlan', 'Breast Cancer Early Detection', 'Research', 'Reducing your breast cancer risk with medication (pre-menopause)', 'Reducing your breast cancer risk with medication (pre-menopause)', 'Reducing your breast cancer risk with medication (post-menopause)', 'How to prepare for menopause', 'How to cope with menopause', 'Reducing your ovarian cancer risk with surgery', 'Lifestyle Factors', 'General Advice', 'Impact of risk reducing mastectomy', 'Impact of risk reducing salpingo-oophorectomy', 'Transgender and Gender Diverse People', 'Family Communication', 'General Issues', 'Managing Pregnancy', 'Fertility', 'Reproductive Genetic Testing', 'Grief Support', 'Less Common Cancers'], 'distinct_count': 20, 'distinct_list': [], 'distinct': ['Family Communication', 'How to prepare for menopause', 'Transgender and Gender Diverse People', 'Breast Cancer Early Detection', 'Less Comm

In [ ]:
pip install chromadb


In [5]:
import chromadb
import uuid
import chromadb.utils.embedding_functions as embedding_functions
import pandas as pd

chroma_client = chromadb.Client(  )
chroma_client = chromadb.PersistentClient(path="./test_db")

chunk = []
metadata = []
for chunked_docs in rs:
    chunk.append(chunked_docs["content"]) 

    chunked_docs["topic"]
    chunked_docs["min_age"]
    chunked_docs["min_age"]
    
    temp_metadata = {
        "topic": chunked_docs["topic"],
        "min_age": chunked_docs["min_age"],
        "max_age": chunked_docs["max_age"]
    }
    
    metadata.append(temp_metadata)
    

openai_ef = embedding_functions.OpenAIEmbeddingFunction(
                api_key="",
                model_name="text-embedding-3-small"
            )

collection = chroma_client.get_or_create_collection(name="my_collection")
collection.add(
    documents= chunk,
    ids=[str(uuid.uuid4()) for _ in chunk],
    metadatas= metadata,
)

results = collection.query(
    query_texts=["what is BRCA1?"], # Chroma will embed this for you
    n_results=3 # how many results to return
)
print(results)

ModuleNotFoundError: No module named 'chromadb'

In [36]:
def query_collection(collection, query, max_results):
    results = collection.query(query_texts=query, n_results=max_results, include=['distances', 'documents']) 
    
    df = pd.DataFrame({
                'id':results['ids'][0], 
                'content':results.get('documents', [[]])[0],
                'score':results['distances'][0]
                
                })
    
    return df


title_query_result = query_collection(
    collection=collection,
    query="what is BRCA1?",
    max_results=10
)
title_query_result.head()

,id,content,score
0,bcdf6532-1c45-4aa8-a4c6-ab348e75f496,Heading: BRCA-P Trial\nPlease see the brochure...,1.055014
1,aa3b9772-0a66-4eea-8a7e-1d1ee736bd04,Heading: BRCA-P Trial\nPlease see the brochure...,1.055014
2,c52a2545-2871-409c-abab-4ba398451812,Heading: BRCA-P Trial\nPlease see the brochure...,1.055014
3,bc7149b3-be26-4bdf-b0c9-f110f485d574,Heading: Brief overview to risk and management...,1.086127
4,6b9636cb-cdab-4c76-8954-2c69aa923ce7,Heading: Brief overview to risk and management...,1.086127


In [8]:
%pip install tiktoken
import tiktoken

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

num_tokens_from_string("tiktoken is great!", "cl100k_base")

  Using cached tiktoken-0.9.0-cp313-cp313-macosx_11_0_arm64.whl.metadata (6.7 kB)
  Using cached regex-2024.11.6-cp313-cp313-macosx_11_0_arm64.whl.metadata (40 kB)
  Using cached requests-2.32.4-py3-none-any.whl.metadata (4.9 kB)
  Using cached charset_normalizer-3.4.2-cp313-cp313-macosx_10_13_universal2.whl.metadata (35 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2025.6.15-py3-none-any.whl.metadata (2.4 kB)
Using cached tiktoken-0.9.0-cp313-cp313-macosx_11_0_arm64.whl (1.0 MB)
Using cached regex-2024.11.6-cp313-cp313-macosx_11_0_arm64.whl (284 kB)
Using cached requests-2.32.4-py3-none-any.whl (64 kB)
Using cached charset_normalizer-3.4.2-cp313-cp313-macosx_10_13_universal2.whl (199 kB)
Using cached urllib3-2.5.0-py3-none-any.whl (129 kB)
Using cached certifi-2025.6.15-py3-none-any.whl (157 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6/6 [tiktoken]
Note: you may need to restart the kernel to use updated packages.


6